Function Calling :-

AI model’s ability to call external functions, APIs, or execute custom logic based on a user’s request. It enables AI models to interact with databases, retrieve real-time data, and execute predefined tasks, making them more dynamic and useful.

AI calls APIs to fetch live weather, stock prices.AI books appointments, adds events to Google Calendar.AI retrieves customer details, past orders from a database.

https://rapidapi.com/ has so many APIs in it

In [34]:
import openai
import os
import json
import http.client
from dotenv import load_dotenv

load_dotenv() # Load environment variables

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # Retrieve OpenAI API key

if not OPENAI_API_KEY:
    raise ValueError("Error: OPENAI_API_KEY is not loaded. Check your .env file.") # Ensure API key is loaded

openai.api_key = OPENAI_API_KEY

def get_current_weather(city):
    try:
        conn = http.client.HTTPSConnection("ai-weather-by-meteosource.p.rapidapi.com")

        headers = {
            "x-rapidapi-key": "efd579194cmsh0412ba72d13838ap13fbb1jsn23382dcae2d8",
            "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
        }

        # Use city name to get latitude & longitude
        geo_endpoint = f"/find_places?text={city}"
        conn.request("GET", geo_endpoint, headers=headers)
        geo_res = conn.getresponse()
        geo_data = json.loads(geo_res.read().decode("utf-8"))

        if not geo_data:
            return {"error": "City not found"}

        lat, lon = geo_data[0]["lat"], geo_data[0]["lon"]

        # Fetch real-time weather using the correct endpoint
        weather_endpoint = f"/time_machine?lat={lat}&lon={lon}&date=2023-08-24&units=auto"
        conn.request("GET", weather_endpoint, headers=headers)
        weather_res = conn.getresponse()
        weather_data = json.loads(weather_res.read().decode("utf-8"))

        return weather_data

    except Exception as e:
        return {"error": str(e)}
    
print(get_current_weather("Dubai"))

{'lat': '25.07725N', 'lon': '55.30927E', 'elevation': 25, 'timezone': 'Asia/Dubai', 'units': 'metric', 'data': [{'date': '2023-08-24T00:00:00', 'weather': 'partly_sunny', 'summary': 'Partly sunny', 'icon': 4, 'temperature': 32.8, 'feels_like': 42.4, 'wind_chill': 35.5, 'dew_point': 30.4, 'wind': {'speed': 1.1, 'gusts': 1.5, 'angle': 267, 'dir': 'W'}, 'cloud_cover': 42, 'pressure': 1000, 'precipitation': {'total': 0.0, 'type': 'none'}, 'ozone': 286, 'humidity': 87.0}, {'date': '2023-08-24T01:00:00', 'weather': 'partly_sunny', 'summary': 'Partly sunny', 'icon': 4, 'temperature': 32.1, 'feels_like': 42.3, 'wind_chill': 34.7, 'dew_point': 30.3, 'wind': {'speed': 0.7, 'gusts': 1.5, 'angle': 201, 'dir': 'SSW'}, 'cloud_cover': 26, 'pressure': 1000, 'precipitation': {'total': 0.0, 'type': 'none'}, 'ozone': 285, 'humidity': 90.0}, {'date': '2023-08-24T02:00:00', 'weather': 'overcast', 'summary': 'Overcast', 'icon': 7, 'temperature': 31.2, 'feels_like': 39.3, 'wind_chill': 33.8, 'dew_point': 30.

In [38]:
# Define function for OpenAI
functions = [
    {
        "name": "get_current_weather",
        "description": "Fetches real-time weather for a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city name, e.g. Dubai"
                },
            },
            "required": ["city"],
        },
    },
    {
        "name": "get_current_location",
        "description": "Fetches lat and log for a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city name, e.g. Dubai"
                },
            },
            "required": ["city"],
        },
    }
]

In [53]:
messages = [{"role": "user", "content": "What is the temperature in Duubi?"}] # User input to OpenAI

response = openai.ChatCompletion.create( # Send request to OpenAI
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

response_message = response["choices"][0]["message"] # Check if OpenAI requests a function call
print(response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"city\":\"Dubai\"}"
  },
  "refusal": null,
  "annotations": []
}


In [54]:
function_name = response_message["function_call"]["name"]
function_args = json.loads(response_message["function_call"]["arguments"])
# print(get_current_weather("AbuDhabi"))
if function_name == "get_current_weather":
    weather_result = get_current_weather(function_args["city"])

    messages.append({ # appending the role as function and passing function name and in content we are giving the json result from API
        "role": "function",
        "name": function_name,
        "content": json.dumps(weather_result) 
    })

    # Ask OpenAI to format the final response
    final_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    print(final_response["choices"][0]["message"]["content"])

The current temperature in Duubi is 32.8°C.
